<a href="https://colab.research.google.com/github/cardstud/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
# Save file to notebook
from google.colab import files
uploaded = files.upload()

Saving house-votes-84.csv to house-votes-84.csv


In [0]:
# Import data analysis and visualization libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

### Attribute Information:

0. Class Name: 2 (democrat, republican)
1. handicapped-infants: 2 (y,n)
2. water-project-cost-sharing: 2 (y,n)
3. adoption-of-the-budget-resolution: 2 (y,n)
4. physician-fee-freeze: 2 (y,n)
5. el-salvador-aid: 2 (y,n)
6. religious-groups-in-schools: 2 (y,n)
7. anti-satellite-test-ban: 2 (y,n)
8. aid-to-nicaraguan-contras: 2 (y,n)
9. mx-missile: 2 (y,n)
10. immigration: 2 (y,n)
11. synfuels-corporation-cutback: 2 (y,n)
12. education-spending: 2 (y,n)
13. superfund-right-to-sue: 2 (y,n)
14. crime: 2 (y,n)
15. duty-free-exports: 2 (y,n)
16. export-administration-act-south-africa: 2 (y,n)

### Read in data and view it

In [0]:
# Save csv as a Pandas DataFrame: also, noticed header issue so fixed
df= pd.read_csv('house-votes-84.csv', header=None)

In [0]:
# Check head for issues
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
# Check tail
df.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,?,?,?,?,n,y,y,y,n,y
434,republican,n,y,n,y,y,y,n,n,n,y,n,y,y,y,?,n


In [0]:
# Check random sample from df
df.sample(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
350,democrat,n,y,n,n,n,y,y,n,y,n,y,n,n,n,y,y
79,republican,n,n,n,y,y,n,n,n,n,y,n,y,y,y,n,n
304,republican,n,y,n,y,y,y,n,n,n,y,n,y,y,y,n,?
54,democrat,y,y,y,n,n,y,?,y,n,n,y,y,n,y,n,?
368,democrat,n,y,y,n,n,y,y,y,n,y,n,n,n,n,y,y
90,democrat,y,n,y,n,n,n,y,y,y,y,n,n,n,n,y,y
274,republican,y,n,n,y,y,n,y,n,n,y,n,n,n,y,y,y
293,democrat,y,y,y,n,n,y,y,y,y,y,y,n,y,y,y,y


In [0]:
# Check Shape and see if matches websites numbers (site has 435 observations, 17 total attributes -16 + target)
df.shape

(435, 17)

In [0]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
5,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
6,democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
7,republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
8,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
9,democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?


### Clean dataset

In [0]:
# Check for odd characters. Found ?. Replace with NaN values
df =df.replace('?', None)

In [0]:
df[11][0]='n'

In [0]:
# Check head again to see if values replaced. They were.
df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,n,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
2,democrat,n,y,y,y,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,y,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,n,y,y,y,y
5,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
6,democrat,n,y,n,y,y,y,n,n,n,n,n,n,y,y,y,y
7,republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,y,y
8,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
9,democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,n,y


In [0]:
# Find sum of missing values
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
dtype: int64

In [0]:
# Check to see if NaN changed to 'nv'
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,n,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
2,democrat,n,y,y,y,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,y,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,n,y,y,y,y


In [0]:
# Check some summary statistics 
df.describe(exclude='number')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
count,435,435,435,435,435,435,435,435,435,435,435,435,435,435,435,435,435
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,n,y,n,y,y,y,y,n,y,n,n,y,y,n,y
freq,267,243,223,255,250,220,278,248,249,220,218,280,257,222,254,250,343


In [0]:
# Change n/y to binary
df =df.replace('y', 1)
df = df.replace('n', 0)

In [0]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,0,1,0,1,1,1,0,0,0,1,0,1,1,1,0,1
1,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
2,democrat,0,1,1,1,1,1,0,0,0,0,1,0,1,1,0,0
3,democrat,0,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1
4,democrat,1,1,1,0,1,1,0,0,0,0,1,0,1,1,1,1


In [0]:
# Change column names
df.columns = ['class', 'infants', 'water_cost', 'budget', 'fee_freeze', 'aid_elsalvador', 'rel_school', 'satellite', 'aid_contras', 'mx_missle', 'immigration', 'cutback', 'education', 'right_to_sue', 'crime', 'duty_free_ex', 'export_south_africa']

In [0]:
df.head()

,class,infants,water_cost,budget,fee_freeze,aid_elsalvador,rel_school,satellite,aid_contras,mx_missle,immigration,cutback,education,right_to_sue,crime,duty_free_ex,export_south_africa
0,republican,0,1,0,1,1,1,0,0,0,1,0,1,1,1,0,1
1,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
2,democrat,0,1,1,1,1,1,0,0,0,0,1,0,1,1,0,0
3,democrat,0,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1
4,democrat,1,1,1,0,1,1,0,0,0,0,1,0,1,1,1,1


In [0]:
df.describe()

,infants,water_cost,budget,fee_freeze,aid_elsalvador,rel_school,satellite,aid_contras,mx_missle,immigration,cutback,education,right_to_sue,crime,duty_free_ex,export_south_africa
count,435.000000,435.000000,435.000000,435.000000,435.000000,435.00000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000,435.000000
mean,0.441379,0.487356,0.586207,0.425287,0.505747,0.63908,0.570115,0.572414,0.494253,0.501149,0.356322,0.409195,0.510345,0.583908,0.425287,0.788506
std,0.497123,0.500416,0.493079,0.494956,0.500543,0.48082,0.495630,0.495298,0.500543,0.500574,0.479463,0.492252,0.500469,0.493477,0.494956,0.408838
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Subset data into 2 subset for democrats and republicans

In [0]:
df_republican = df[df['class']== 'republican']

In [0]:
df_republican.shape


(168, 17)

In [0]:
df_republican.columns

Index(['class', 'infants', 'water_cost', 'budget', 'fee_freeze',
       'aid_elsalvador', 'rel_school', 'satellite', 'aid_contras', 'mx_missle',
       'immigration', 'cutback', 'education', 'right_to_sue', 'crime',
       'duty_free_ex', 'export_south_africa'],
      dtype='object')

In [0]:
df_democrat = df[df['class']== 'democrat']

In [0]:
df_democrat.shape

(267, 17)

In [0]:
df.shape

(435, 17)

In [0]:
df_democrat.describe()

,infants,water_cost,budget,fee_freeze,aid_elsalvador,rel_school,satellite,aid_contras,mx_missle,immigration,cutback,education,right_to_sue,crime,duty_free_ex,export_south_africa
count,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000
mean,0.599251,0.490637,0.872659,0.071161,0.224719,0.475655,0.767790,0.827715,0.734082,0.471910,0.494382,0.146067,0.295880,0.348315,0.625468,0.865169
std,0.490971,0.500851,0.333980,0.257576,0.418181,0.500345,0.423035,0.378337,0.442650,0.500148,0.500907,0.353837,0.457295,0.477331,0.484911,0.342185
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
df_republican.describe()

,infants,water_cost,budget,fee_freeze,aid_elsalvador,rel_school,satellite,aid_contras,mx_missle,immigration,cutback,education,right_to_sue,crime,duty_free_ex,export_south_africa
count,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000
mean,0.190476,0.482143,0.130952,0.988095,0.952381,0.898810,0.255952,0.166667,0.113095,0.547619,0.136905,0.827381,0.851190,0.958333,0.107143,0.666667
std,0.393851,0.501175,0.338357,0.108782,0.213596,0.302482,0.437700,0.373792,0.317656,0.499215,0.344774,0.379047,0.356965,0.200424,0.310219,0.472814
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
df_republican.dtypes

class                  object
infants                 int64
water_cost              int64
budget                  int64
fee_freeze              int64
aid_elsalvador          int64
rel_school              int64
satellite               int64
aid_contras             int64
mx_missle               int64
immigration             int64
cutback                 int64
education               int64
right_to_sue            int64
crime                   int64
duty_free_ex            int64
export_south_africa     int64
dtype: object

In [0]:
df_republican.head()

,class,infants,water_cost,budget,fee_freeze,aid_elsalvador,rel_school,satellite,aid_contras,mx_missle,immigration,cutback,education,right_to_sue,crime,duty_free_ex,export_south_africa
0,republican,0,1,0,1,1,1,0,0,0,1,0,1,1,1,0,1
1,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
7,republican,0,1,0,1,1,1,0,0,0,0,0,0,1,1,1,1
8,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
10,republican,0,1,0,1,1,0,0,0,0,0,0,0,1,1,0,0


### Infant issue

In [0]:
# import ttest
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the infant issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the infant issue

In [0]:
# Confirm stats
rep_infants = df_republican.groupby('class')['infants'].mean()
rep_infants

class
republican    0.190476
Name: infants, dtype: float64

In [0]:
demo_infants = (df_democrat.groupby('class')['infants'].mean())
demo_infants

class
democrat    0.599251
Name: infants, dtype: float64

In [0]:
rep_infants = df_republican['infants']

In [0]:
rep_infants.sum()

32

In [0]:
rep_infants.count()

168

In [0]:
demo_infants = df_democrat['infants']

In [0]:
demo_infants.sum()

160

In [0]:
demo_infants.count()

267

In [0]:
ttest_ind(rep_infants, demo_infants)

Ttest_indResult(statistic=-9.103596896264648, pvalue=3.229224805329127e-18)

In [0]:
# Based on this, reject the null hypothesis that both parties voted the same on the infants issue

### Water Cost issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the water cost issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the water cost issue

In [0]:
rep_water_cost = df_republican['water_cost']
rep_water_cost.head()

0     1
1     1
7     1
8     1
10    1
Name: water_cost, dtype: int64

In [0]:
rep_water_cost.sum()

81

In [0]:
rep_water_cost.count()

168

In [0]:
rep_infants = df_republican['water_cost']

In [0]:
demo_water_cost=df_democrat['water_cost']
demo_water_cost.head()

2    1
3    1
4    1
5    1
6    1
Name: water_cost, dtype: int64

In [0]:
demo_water_cost.sum()

131

In [0]:
demo_water_cost.count()

267

In [0]:
demo_water_cost = df_democrat['water_cost']

In [0]:
ttest_ind(rep_water_cost, demo_water_cost)

Ttest_indResult(statistic=-0.17216828566462639, pvalue=0.863385668888899)

In [0]:
# Based on this, we fail to reject the null hypothesis that both parties voted the same on the water_cost issue

### Budget issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Budget issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Budget issue

In [0]:
rep_budget = df_republican.groupby('class')['budget'].mean()
rep_budget

class
republican    0.130952
Name: budget, dtype: float64

In [0]:
rep_budget = df_republican['budget']
rep_budget.head()

0     0
1     0
7     0
8     0
10    0
Name: budget, dtype: int64

In [0]:
print(rep_budget.sum(), ' ', rep_budget.count())

22   168


In [0]:
demo_budget = df_democrat.groupby('class')['budget'].mean()
demo_budget

class
democrat    0.872659
Name: budget, dtype: float64

In [0]:
demo_budget = df_democrat['budget']
demo_budget.head()

2    1
3    1
4    1
5    1
6    0
Name: budget, dtype: int64

In [0]:
print(demo_budget.sum(), ' ', demo_budget.count())

233   267


In [0]:
ttest_ind(rep_budget, demo_budget)

Ttest_indResult(statistic=-22.437726247750923, pvalue=1.550290571202001e-74)

In [0]:
# Based on this, we reject the null hypothesis

### Fee freeze issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Fee freeze issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Fee freeze issue

In [0]:
rep_fee_freeze = df_republican.groupby('class')['fee_freeze'].mean()
rep_fee_freeze

class
republican    0.988095
Name: fee_freeze, dtype: float64

In [0]:
rep_fee_freeze = df_republican['fee_freeze']
rep_fee_freeze.head()

0     1
1     1
7     1
8     1
10    1
Name: fee_freeze, dtype: int64

In [0]:
print(rep_fee_freeze.sum(), ' ' , rep_fee_freeze.count())

166   168


In [0]:
demo_fee_freeze = df_democrat.groupby('class')['fee_freeze'].mean()
demo_fee_freeze

class
democrat    0.071161
Name: fee_freeze, dtype: float64

In [0]:
demo_fee_freeze = df_democrat['fee_freeze']

In [0]:
ttest_ind(rep_fee_freeze, demo_fee_freeze)

Ttest_indResult(statistic=43.73735799798205, pvalue=5.6609202660069e-161)

In [0]:
# Based on above, we reject the null hypothesis

### Aid El Salvador issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Aid to El Salvador issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Aid to El Salvador issue

In [0]:
rep_aid_elsal = df_republican.groupby('class')['aid_elsalvador'].mean()
rep_aid_elsal

class
republican    0.952381
Name: aid_elsalvador, dtype: float64

In [0]:
rep_aid_elsal = df_republican['aid_elsalvador']

In [0]:
print(rep_aid_elsal.sum(), ' ' , rep_aid_elsal.count())

160   168


In [0]:
demo_aid_elsal = df_democrat.groupby('class')['aid_elsalvador'].mean()
demo_aid_elsal

class
democrat    0.224719
Name: aid_elsalvador, dtype: float64

In [0]:
demo_aid_elsal = df_democrat['aid_elsalvador']

In [0]:
print(demo_aid_elsal.sum(), ' ' , demo_aid_elsal.count())

60   267


In [0]:
ttest_ind(rep_aid_elsal, demo_aid_elsal)

Ttest_indResult(statistic=20.897604777126915, pvalue=1.435893650544929e-67)

In [0]:
# based on above, we reject the null hypothesis

### Religious groups in school issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Aid to El Salvador issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Aid to El Salvador issue

In [0]:
rep_rel_school = df_republican.groupby('class')['rel_school'].mean()
rep_rel_school

class
republican    0.89881
Name: rel_school, dtype: float64

In [0]:
rep_rel_school = df_republican['rel_school']

In [0]:
print(rep_rel_school.sum(), ' ' , rep_rel_school.count())

151   168


In [0]:
demo_rel_school = df_democrat.groupby('class')['rel_school'].mean()
demo_rel_school

class
democrat    0.475655
Name: rel_school, dtype: float64

In [0]:
demo_rel_school = df_democrat['rel_school']

In [0]:
print(demo_rel_school.sum(), ' ' , demo_rel_school.count())

127   267


In [0]:
ttest_ind(rep_rel_school, demo_rel_school)

Ttest_indResult(statistic=9.881928443377337, pvalue=6.6819807113738e-21)

In [0]:
# We reject the null hypothesis

### Satellite issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Satellite issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Satellite issue

In [0]:
rep_satellite = df_republican.groupby('class')['satellite'].mean()
rep_satellite

class
republican    0.255952
Name: satellite, dtype: float64

In [0]:
rep_satellite = df_republican['satellite']

In [0]:
print(rep_satellite.sum(), ' ', rep_satellite.count())

43   168


In [0]:
demo_satellite = df_democrat.groupby('class')['satellite'].mean()
demo_satellite

class
democrat    0.76779
Name: satellite, dtype: float64

In [0]:
demo_satellite = df_democrat['satellite']

In [0]:
print(demo_satellite.sum(), ' ' , demo_satellite.count())

205   267


In [0]:
ttest_ind(rep_satellite, demo_satellite)

Ttest_indResult(statistic=-12.122537793016981, pvalue=2.530872694121891e-29)

In [0]:
# we reject the null hypothesis

### Aid to Contras issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Aid to Contras issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Aid to Contras issue

In [0]:
rep_aid_contras = df_republican.groupby('class')['aid_contras'].mean()
rep_aid_contras

class
republican    0.166667
Name: aid_contras, dtype: float64

In [0]:
rep_aid_contras = df_republican['aid_contras']

In [0]:
print(rep_aid_contras.sum(), ' ' , rep_aid_contras.count())

28   168


In [0]:
demo_aid_contras = df_democrat.groupby('class')['aid_contras'].mean()
demo_aid_contras

class
democrat    0.827715
Name: aid_contras, dtype: float64

In [0]:
demo_aid_contras = df_democrat['aid_contras']

In [0]:
print(demo_aid_contras.sum(), ' ', demo_aid_contras.count())

221   267


In [0]:
ttest_ind(rep_aid_contras, demo_aid_contras)

Ttest_indResult(statistic=-17.825007262252463, pvalue=1.0619157607124256e-53)

In [0]:
# We reject the null hypothesis

### MX Missile issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Mx Missile issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Mx Missile issue

In [0]:
rep_mx_missile = df_republican.groupby('class')['mx_missle'].mean()
rep_mx_missile

class
republican    0.113095
Name: mx_missle, dtype: float64

In [0]:
rep_mx_missile = df_republican['mx_missle']

In [0]:
print(rep_mx_missile.sum(), ' ' , rep_mx_missile.count())

19   168


In [0]:
demo_mx_missile = df_democrat.groupby('class')['mx_missle'].mean()
demo_mx_missile

class
democrat    0.734082
Name: mx_missle, dtype: float64

In [0]:
demo_mx_missile = df_democrat['mx_missle']

In [0]:
print(demo_mx_missile.sum(), ' ' , demo_mx_missile.count())

196   267


In [0]:
ttest_ind(rep_mx_missile, demo_mx_missile)

Ttest_indResult(statistic=-15.80006310521197, pvalue=9.931457737101551e-45)

In [0]:
# We reject the null hypothesis

### Immigration issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Immigration issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Immigration issue

In [0]:
rep_immigration = df_republican.groupby('class')['immigration'].mean()
rep_immigration

class
republican    0.547619
Name: immigration, dtype: float64

In [0]:
rep_immigration = df_republican['immigration']

In [0]:
print(rep_immigration.sum(), ' ', rep_immigration.count())

92   168


In [0]:
demo_immigration = df_democrat.groupby('class')['immigration'].mean()
demo_immigration

class
democrat    0.47191
Name: immigration, dtype: float64

In [0]:
demo_immigration = df_democrat['immigration']

In [0]:
print(demo_immigration.sum(), ' ', demo_immigration.count())

126   267


In [0]:
ttest_ind(rep_immigration, demo_immigration)

Ttest_indResult(statistic=1.5382490514959393, pvalue=0.12471814090276395)

In [0]:
# We fail to reject the null hypothesis

### Cutback issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Cutback issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Cutback issue

In [0]:
rep_cutback = df_republican.groupby('class')['cutback'].mean()
rep_cutback

class
republican    0.136905
Name: cutback, dtype: float64

In [0]:
rep_cutback = df_republican['cutback']

In [0]:
print(rep_cutback.sum(), ' ' , rep_cutback.count())

23   168


In [0]:
demo_cutback = df_democrat.groupby('class')['cutback'].mean()
demo_cutback

class
democrat    0.494382
Name: cutback, dtype: float64

In [0]:
demo_cutback = df_democrat['cutback']

In [0]:
print(demo_cutback.sum(), ' ', demo_cutback.count())

132   267


In [0]:
ttest_ind(rep_cutback, demo_cutback)

Ttest_indResult(statistic=-8.117404958997604, pvalue=4.9872928351929665e-15)

In [0]:
# we reject teh null hypothesis

### Education issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Education issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Education issue

In [0]:
rep_education = df_republican.groupby('class')['education'].mean()
rep_education

class
republican    0.827381
Name: education, dtype: float64

In [0]:
rep_education = df_republican['education']

In [0]:
print(rep_education.sum(), ' ', rep_education.count())

139   168


In [0]:
demo_education = df_democrat.groupby('class')['education'].mean()
demo_education

class
democrat    0.146067
Name: education, dtype: float64

In [0]:
demo_education = df_democrat['education']

In [0]:
print(demo_education.sum(), ' ' , demo_education.count())

39   267


In [0]:
ttest_ind(rep_education, demo_education)

Ttest_indResult(statistic=19.019061449784374, pvalue=4.527822625602129e-59)

In [0]:
# We reject the null hypothesis

### Right to sue issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Right to Sue issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Right to Sue issue

In [0]:
rep_right_sue = df_republican.groupby('class')['right_to_sue'].mean()
rep_right_sue

class
republican    0.85119
Name: right_to_sue, dtype: float64

In [0]:
rep_right_sue = df_republican['right_to_sue']

In [0]:
print(rep_right_sue.sum(), ' ', rep_right_sue.count())

143   168


In [0]:
demo_right_sue = df_democrat.groupby('class')['right_to_sue'].mean()
demo_right_sue

class
democrat    0.29588
Name: right_to_sue, dtype: float64

In [0]:
demo_right_sue = df_democrat['right_to_sue']

In [0]:
print(demo_right_sue.sum(), ' ', demo_right_sue.count())

79   267


In [0]:
ttest_ind(rep_right_sue, demo_right_sue)

Ttest_indResult(statistic=13.380347818047976, pvalue=2.0452746562067425e-34)

In [0]:
# Reject the null hypothesis

### Crime issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Crime issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Crime issue

In [0]:
rep_crime = df_republican.groupby('class')['crime'].mean()
rep_crime

class
republican    0.958333
Name: crime, dtype: float64

In [0]:
rep_crime = df_republican['crime']

In [0]:
print(rep_crime.sum(), ' ', rep_crime.count())

161   168


In [0]:
demo_crime = df_democrat.groupby('class')['crime'].mean()
demo_crime

class
democrat    0.348315
Name: crime, dtype: float64

In [0]:
demo_crime = df_democrat['crime']

In [0]:
print(demo_crime.sum(), ' ', demo_crime.count())

93   267


In [0]:
ttest_ind(rep_crime, demo_crime)

Ttest_indResult(statistic=15.710738207208037, pvalue=2.438075126832831e-44)

In [0]:
# reject the null hypothesis

### Duty free export issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Crime issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Crime issue

In [0]:
rep_duty_free = df_republican.groupby('class')['duty_free_ex'].mean()
rep_duty_free

class
republican    0.107143
Name: duty_free_ex, dtype: float64

In [0]:
rep_duty_free = df_republican['duty_free_ex']

In [0]:
print(rep_duty_free.sum(), ' ', rep_duty_free.count())

18   168


In [0]:
demo_duty_free = df_democrat.groupby('class')['duty_free_ex'].mean()
demo_duty_free

class
democrat    0.625468
Name: duty_free_ex, dtype: float64

In [0]:
demo_duty_free = df_democrat['duty_free_ex']

In [0]:
print(demo_duty_free.sum(), ' ', demo_duty_free.count())

167   267


In [0]:
ttest_ind(rep_duty_free, demo_duty_free)

Ttest_indResult(statistic=-12.35237116310192, pvalue=3.0884255458450424e-30)

In [0]:
# Reject null hypothesis

### Export South Africa issue

In [0]:
# Null hypothesis:  Republicans and Democrats voted the same on the Crime issue
# alternative hypothesis: Republicans and Democrats did not vote the same on the Crime issue

In [0]:
rep_export_sa = df_republican.groupby('class')['export_south_africa'].mean()
rep_export_sa

class
republican    0.666667
Name: export_south_africa, dtype: float64

In [0]:
rep_export_sa = df_republican['export_south_africa']

In [0]:
print(rep_export_sa.sum(), ' ' , rep_export_sa.count())

112   168


In [0]:
demo_export_sa = df_democrat.groupby('class')['export_south_africa'].mean()
demo_export_sa

class
democrat    0.865169
Name: export_south_africa, dtype: float64

In [0]:
demo_export_sa = df_democrat['export_south_africa']

In [0]:
print(demo_export_sa.sum(), ' ', demo_export_sa.count())

231   267


In [0]:
ttest_ind(rep_export_sa, demo_export_sa)

Ttest_indResult(statistic=-5.068672413760907, pvalue=5.946671321445181e-07)

In [0]:
# Reject null hypothesis

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.


In [0]:
# Goal 1: Completed above
# Goal 2: Exports to South Africa is supported by more by Democrats than Republicans          -pvalue=5.946671321445181e-07
# Goal 3: Fee Freeze issue is supported by more Republicans than Democrats                    -pvalue=5.6609202660069e-161)
# Goal 4: Water cost issue showed not much difference between Democrats and Republicans       -pvalue=0.863385668888899